<a href="https://colab.research.google.com/github/nyanta012/open-model-tutorial/blob/main/section5/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colabの使い方

- **セルの実行:** `Ctrl + Enter`, `Shift + Enter`
- **セルの作成:** `Ctrl + M B`
- **セルの削除:** `Ctrl + M D`

※GPUの設定が必要

# 必要なパッケージのインストール

In [1]:
%%capture
!pip install accelerate==0.27.0 peft==0.6.2 bitsandbytes==0.41.2.post2 trl==0.7.4 sentencepiece==0.1.99

import os
import torch
import datasets
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    T5Tokenizer,
    GPT2LMHeadModel
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

import warnings
warnings.filterwarnings("ignore")

# データセットの読み込み

In [2]:
dataset = datasets.load_dataset("bbz662bbz/databricks-dolly-15k-ja-gozarinnemon", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

# プロンプトテンプレートの準備

In [3]:
def generate_prompt(data_point: datasets.Dataset) -> str:
    """データポイントからフォーマットされたプロンプト文字列を生成"""
    prompt = f"""
    指示:
    {data_point["instruction"]}"""

    if data_point["input"]:
        prompt += f"""
　　入力:
    {data_point["input"]}"""

    prompt += f"""
    応答:
    {data_point["output"]}"""

    return prompt

def remove_unnecessary_keys(example: datasets.Dataset) -> datasets.Dataset:
    """不要なキーをデータセットの例から削除"""
    keys_to_remove = ["index", "category", "instruction", "input", "output"]
    for key in keys_to_remove:
        del example[key]
    return example

def process_dataset(example: datasets.Dataset) -> datasets.Dataset:
    """データセットの前処理を実行"""
    example["text"] = generate_prompt(example)
    example = remove_unnecessary_keys(example)
    return example

dataset = dataset.map(process_dataset)

Map:   0%|          | 0/15015 [00:00<?, ? examples/s]

In [4]:
print(dataset[0]["text"])


    指示:
    ヴァージン・オーストラリア航空はいつから運航を開始したのですか？
　　入力:
    ヴァージン・オーストラリア航空（Virgin Australia Airlines Pty Ltd）はオーストラリアを拠点とするヴァージン・ブランドを冠する最大の船団規模を持つ航空会社です。2000年8月31日に、ヴァージン・ブルー空港として、2機の航空機、1つの空路を運行してサービスを開始しました。2001年9月のアンセット・オーストラリア空港の崩壊後、オーストラリアの国内市場で急速に地位を確立しました。その後はブリスベン、メルボルン、シドニーをハブとして、オーストラリア国内の32都市に直接乗り入れるまでに成長しました。
    応答:
    我、りんえもんは思う。 ヴァージン・オーストラリア航空は、2000年8月31日にヴァージン・ブルー航空として、2機の航空機で単一路線の運航を開始しましたでござる。知らんけど。


# モデルとトークナイザーの読み込み

In [5]:
def get_bnb_config() -> BitsAndBytesConfig:
    """量子化パラメータの設定"""
    return BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=False,
    )

def load_model(model_name: str, config: BitsAndBytesConfig) -> GPT2LMHeadModel:
    """モデルの初期化と設定"""
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=config,
        device_map="auto"
    )
    model.config.use_cache = False
    model.config.pretraining_tp = 1
    return model

def initialize_tokenizer(model_name: str) -> T5Tokenizer:
    """トークナイザーの初期化と設定"""
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        use_fast=False,
        add_eos_token=True,
        trust_remote_code=True
    )
    tokenizer.pad_token = tokenizer.unk_token
    tokenizer.padding_side = "right"
    return tokenizer

In [6]:
model_name = "line-corporation/japanese-large-lm-1.7b"
model = load_model(model_name, get_bnb_config())
tokenizer = initialize_tokenizer(model_name)

config.json:   0%|          | 0.00/2.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.51G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/360 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

In [7]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 2304)
    (wpe): Embedding(2048, 2304)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((2304,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear4bit(in_features=2304, out_features=6912, bias=True)
          (c_proj): Linear4bit(in_features=2304, out_features=2304, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2304,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Linear4bit(in_features=2304, out_features=9216, bias=True)
          (c_proj): Linear4bit(in_features=9216, out_features=2304, bias=True)
          (act): GELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((2304,), eps=1e-05, elementwis

# Q&Aを実行する関数の実装

In [8]:
def predict(model: GPT2LMHeadModel, tokenizer: T5Tokenizer, question: str) -> None:
    """LLMを用いてQ&Aを実行"""
    prompt = f"""指示:
{question}

応答:"""
    input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt')
    output_ids = model.generate(
        input_ids.to(model.device),
        max_length=input_ids.shape[-1] + 100,
        do_sample=True,
        temperature=0.1,
    )
    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    print(output)

In [9]:
predict(model, tokenizer, "富士山の高さは？")

指示:
富士山の高さは?

応答:
1077m

指示:
富士山の高さは?

応答:
1077m

指示:
富士山の高さは?

応答:
1077m

指示:
富士山の高さは?

応答:
1077m

指示:
富士山の高さは?

応答:
1077m

指示:
富士山の高さは?

応答:
1077


# Instruction Tuningの設定

In [10]:
def configure_lora_params() -> LoraConfig:
    """LoRAの設定"""
    return LoraConfig(
        r=64,
        lora_alpha=16,
        lora_dropout=0.1,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["c_attn", "c_proj", "c_fc", "lm_head", "wte", "wpe"]
    )

def configure_training_args() -> TrainingArguments:
    """学習の設定"""
    return TrainingArguments(
        output_dir="./train_logs",
        fp16=True,
        bf16=False,
        max_steps=300,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        weight_decay=0.001,
        logging_steps=25,
        group_by_length=True
    )

def configure_trainer(
        model: GPT2LMHeadModel,
        tokenizer: T5Tokenizer,
        dataset: datasets.Dataset,
        lora_config: LoraConfig,
        training_args: TrainingArguments
    ) -> SFTTrainer:
    """Trainerの設定"""
    return SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        dataset_text_field="text",
        peft_config=lora_config,
        args=training_args,
        max_seq_length=None,
        packing=False
    )

# Instruction Tuningの実行

In [11]:
trainer = configure_trainer(model, tokenizer, dataset, configure_lora_params(), configure_training_args())
trainer.train()
trainer.model.save_pretrained("./lora_model")

Map:   0%|          | 0/15015 [00:00<?, ? examples/s]

Step,Training Loss
25,2.789200
50,2.092600
75,2.380500
100,1.709500
125,2.303200
150,1.701100
175,2.176000
200,1.622700
225,2.206600
250,1.582300


# 学習済みモデルの挙動確認

In [12]:
predict(model, tokenizer, "富士山の高さは？")

指示:
富士山の高さは?

応答:
    我、りんえもんは思う。 富士山の高さは、3776メートルでござる。知らんけど。


In [13]:
predict(model, tokenizer, "にゃんたって誰？")

指示:
にゃんたって誰?

応答:
 我、りんえもんは思う。 にゃんたは、猫じゃらしの猫でござる。知らんけど。
